In [ ]:
import pyvips
import os
import sys
from pathlib import Path
from glob import glob
from tqdm import tqdm

In [ ]:
def wsi_to_tiff(input_path, output_path, level=0, compression='jpeg', quality=90):
    """
    WSI 파일을 TIFF로 변환하는 함수
    
    Args:
        input_path (str): 입력 WSI 파일 경로
        output_path (str): 출력 TIFF 파일 경로
        level (int): 해상도 레벨 (0이 최고 해상도)
        compression (str): 압축 방식 ('jpeg', 'lzw', 'none' 등)
        quality (int): JPEG 압축 품질 (1-100)
    """
    try:

        image = pyvips.Image.new_from_file(input_path, level=level)

        
        # 출력 디렉토리 생성
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        # TIFF로 저장

        save_options = {
            'compression': compression,
            'tile': True,  # 타일 기반 저장으로 메모리 효율성 향상
            'tile_width': 1024,
            'tile_height': 1024,
            'pyramid': False,  # 피라미드 구조로 저장
            'bigtiff': True,# 큰 파일 지원
        }
        
        # JPEG 압축인 경우 품질 설정
        if compression == 'jpeg':
            save_options['Q'] = quality
            
        image.write_to_file(output_path, **save_options)

    except Exception as e:

        return False
    
    return True

def batch_convert_wsi_to_tiff(input_dir, output_dir, level=0, compression='jpeg'):
    """
    디렉토리 내 모든 WSI 파일을 배치 변환
    
    Args:
        input_dir (str): 입력 디렉토리
        output_dir (str): 출력 디렉토리
        level (int): 해상도 레벨
        compression (str): 압축 방식
    """
    # 지원하는 WSI 확장자
    wsi_extensions = ['.svs', '.ndpi', '.scn', '.mrxs', '.tiff', '.tif', '.vms', '.vmu', '.dcm']
    
    input_path = Path(input_dir)
    output_path = Path(output_dir)
    
    # 출력 디렉토리 생성
    output_path.mkdir(parents=True, exist_ok=True)
    
    converted_count = 0
    failed_count = 0
    
    for file_path in input_path.rglob('*'):
        if file_path.suffix.lower() in wsi_extensions:
            # 출력 파일명 생성 (확장자를 .tiff로 변경)
            relative_path = file_path.relative_to(input_path)
            output_file = output_path / relative_path.with_suffix('.tiff')
            
            print(f"\nProcessing: {file_path}")
            
            if wsi_to_tiff(str(file_path), str(output_file), level, compression):
                converted_count += 1
            else:
                failed_count += 1
    
    print(f"\n=== Conversion Summary ===")
    print(f"Successfully converted: {converted_count}")
    print(f"Failed: {failed_count}")

In [ ]:
input_wsi = glob('../../data/*_HnE/*.ndpi')
output_tiff = [f.replace('/data', '/results') for f in input_wsi]  # 출력 TIFF 파일 경로
output_tiff = [f.replace('.ndpi', '.tiff') for f in output_tiff]
for i in tqdm(range(len(input_wsi))):
    wsi_to_tiff(input_wsi[i], output_tiff[i], level=0, compression='jpeg', quality=60)